In [1]:
import os
os.chdir('..')

from config import *
from utilities import get_model_name
from dataset import ClassificationDataset

import functools
import numpy as np
import pandas as pd

results_dict = {}
internal, external = 11, 12
encode_method = "dummy"
impute_method = "itenet"
fs_method, fs_ratio = "dnp", 50/142
norm_method = "maxmin"
classification_dataset = ClassificationDataset(
    internal, external, encode_method, impute_method, fs_method, fs_ratio, norm_method, random_state=SEED)

In [2]:
from utilities import GNNClassifier

model_fn = functools.partial(
    GNNClassifier, random_state=SEED, max_iter=50,
    momentum=0.9, squares=0.999, optimizer_t='sgdm',
    use_residual=True, use_batch_norm=False)
param_grid_list = [{
    'hidden_layer_sizes': [(128, 128), (64, 64, 64)],
    'learning_rate': [5e-3, 1e-2, 2e-2],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

GNNClassifier
model(hidden_layer_sizes=(128, 128), learning_rate=0.005)
Valid:         AUC: 0.9096 | AUPR: 0.5957 | Acc: 94.87%
model(hidden_layer_sizes=(128, 128), learning_rate=0.01)
Valid:         AUC: 0.9161 | AUPR: 0.6159 | Acc: 95.08%
model(hidden_layer_sizes=(128, 128), learning_rate=0.02)
Valid:         AUC: 0.9186 | AUPR: 0.6272 | Acc: 95.13%
model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.005)
Valid:         AUC: 0.9116 | AUPR: 0.5950 | Acc: 94.92%
model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.01)
Valid:         AUC: 0.9169 | AUPR: 0.6171 | Acc: 95.11%
model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.02)
Valid:         AUC: 0.9203 | AUPR: 0.6324 | Acc: 95.24%
best model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.02)
Valid:         AUC: 0.9203 | AUPR: 0.6324 | Acc: 95.24%
Internal test: AUC: 0.9229 | AUPR: 0.6670 | Acc: 94.89%
External test: AUC: 0.9175 | AUPR: 0.6196 | Acc: 94.71%


In [3]:
# from sklearn.neural_network import MLPClassifier
from utilities import MLPClassifier

model_fn = functools.partial(
    MLPClassifier, random_state=SEED, max_iter=50,
    momentum=0.9, squares=0.999, optimizer_t='sgdm',
    use_residual=True, use_batch_norm=False)
param_grid_list = [{
    # hidden_layer_sizes = (64, 32)
    # config       lr_range            worst_auc
    # adam w/ bn:  [2e-5, 5e-5, 1e-4]  0.8942
    # adam w/o bn: [5e-5, 1e-4, 2e-4]  0.9148
    # sgdm w/ bn:  [1e-3, 2e-3, 5e-3]  0.8982
    # sgdm w/o bn: [5e-3, 1e-2, 2e-2]  0.9177

    'hidden_layer_sizes': [(128, 128), (64, 64, 64)],
    'learning_rate': [5e-3, 1e-2, 2e-2],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

MLPClassifier
model(hidden_layer_sizes=(128, 128), learning_rate=0.005)
Valid:         AUC: 0.9076 | AUPR: 0.5896 | Acc: 94.84%
model(hidden_layer_sizes=(128, 128), learning_rate=0.01)
Valid:         AUC: 0.9152 | AUPR: 0.6158 | Acc: 95.10%
model(hidden_layer_sizes=(128, 128), learning_rate=0.02)
Valid:         AUC: 0.9177 | AUPR: 0.6279 | Acc: 95.11%
model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.005)
Valid:         AUC: 0.9098 | AUPR: 0.5853 | Acc: 94.74%
model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.01)
Valid:         AUC: 0.9148 | AUPR: 0.6076 | Acc: 95.04%
model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.02)
Valid:         AUC: 0.9182 | AUPR: 0.6224 | Acc: 95.07%
best model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.02)
Valid:         AUC: 0.9182 | AUPR: 0.6224 | Acc: 95.07%
Internal test: AUC: 0.9199 | AUPR: 0.6644 | Acc: 95.16%
External test: AUC: 0.9196 | AUPR: 0.6293 | Acc: 94.87%


In [4]:
# from sklearn.exceptions import ConvergenceWarning
# import warnings
# warnings.filterwarnings("ignore", category=ConvergenceWarning)
from sklearn.linear_model import LogisticRegression

model_fn = functools.partial(
    LogisticRegression, random_state=SEED, n_jobs=-1, 
    max_iter=2000, solver='saga', penalty='l2',
    )
param_grid_list = [{
    'C': [1.0, 10.0, 100.0],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(
    model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(
    best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

LogisticRegression
model(C=1.0)
Valid:         AUC: 0.9151 | AUPR: 0.6058 | Acc: 95.01%
model(C=10.0)
Valid:         AUC: 0.9150 | AUPR: 0.6060 | Acc: 94.98%
model(C=100.0)
Valid:         AUC: 0.9150 | AUPR: 0.6055 | Acc: 94.98%
best model(C=1.0)
Valid:         AUC: 0.9151 | AUPR: 0.6058 | Acc: 95.01%
Internal test: AUC: 0.9160 | AUPR: 0.6465 | Acc: 94.96%
External test: AUC: 0.9139 | AUPR: 0.5891 | Acc: 94.68%


In [5]:
import warnings
from pytorch_tabnet.tab_model import TabNetClassifier
warnings.filterwarnings("ignore", category=UserWarning)

model_fn = functools.partial(
    TabNetClassifier, seed=SEED, 
    n_d=4, n_a=4, momentum=0.05, 
    n_steps=3, gamma=1.75, cat_emb_dim=1,
    n_independent=2, n_shared=2,
    verbose=0)
param_grid_list = [{
    'optimizer_params': [dict(lr=lr) for lr in [5e-2]],
    'gamma': [1.5, 1.75, 2.0],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)
warnings.resetwarnings()

TabNetClassifier
model(optimizer_params={'lr': 0.05}, gamma=1.5)
Valid:         AUC: 0.8921 | AUPR: 0.5474 | Acc: 94.24%
model(optimizer_params={'lr': 0.05}, gamma=1.75)
Valid:         AUC: 0.8983 | AUPR: 0.5683 | Acc: 94.90%
model(optimizer_params={'lr': 0.05}, gamma=2.0)
Valid:         AUC: 0.9063 | AUPR: 0.5754 | Acc: 94.76%
best model(optimizer_params={'lr': 0.05}, gamma=2.0)
Valid:         AUC: 0.9063 | AUPR: 0.5754 | Acc: 94.76%
Internal test: AUC: 0.9057 | AUPR: 0.5966 | Acc: 93.21%
External test: AUC: 0.9034 | AUPR: 0.5515 | Acc: 93.40%


In [6]:
from sklearn.ensemble import RandomForestClassifier

model_fn = functools.partial(
    RandomForestClassifier, random_state=SEED, n_jobs=-1,
    ccp_alpha=0.0, class_weight='balanced', max_depth=5, bootstrap=False)
param_grid_list = [{
        'criterion': ['gini', 'log_loss', 'entropy'],
        'n_estimators': [50, 100, 200],
    }
]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

RandomForestClassifier
model(criterion=gini, n_estimators=50)
Valid:         AUC: 0.9066 | AUPR: 0.6204 | Acc: 86.70%
model(criterion=gini, n_estimators=100)
Valid:         AUC: 0.9069 | AUPR: 0.6206 | Acc: 86.93%
model(criterion=gini, n_estimators=200)
Valid:         AUC: 0.9083 | AUPR: 0.6260 | Acc: 86.87%
model(criterion=log_loss, n_estimators=50)
Valid:         AUC: 0.9074 | AUPR: 0.6223 | Acc: 86.41%
model(criterion=log_loss, n_estimators=100)
Valid:         AUC: 0.9077 | AUPR: 0.6236 | Acc: 86.88%
model(criterion=log_loss, n_estimators=200)
Valid:         AUC: 0.9084 | AUPR: 0.6271 | Acc: 86.73%
model(criterion=entropy, n_estimators=50)
Valid:         AUC: 0.9074 | AUPR: 0.6223 | Acc: 86.41%
model(criterion=entropy, n_estimators=100)
Valid:         AUC: 0.9077 | AUPR: 0.6236 | Acc: 86.88%
model(criterion=entropy, n_estimators=200)
Valid:         AUC: 0.9084 | AUPR: 0.6271 | Acc: 86.73%
best model(criterion=log_loss, n_estimators=200)
Valid:         AUC: 0.9084 | AUPR: 0.6271 | Ac

In [7]:
from sklearn.svm import SVC

model_fn = functools.partial(
    SVC, random_state=SEED, probability=True,
    class_weight='balanced')
param_grid_list = [{
    'kernel': ['linear'],
    'C': [0.01, 0.1, 1],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

SVC
model(kernel=linear, C=0.01)
Valid:         AUC: 0.9130 | AUPR: 0.5561 | Acc: 94.40%
model(kernel=linear, C=0.1)
Valid:         AUC: 0.9151 | AUPR: 0.5683 | Acc: 94.52%
model(kernel=linear, C=1)
Valid:         AUC: 0.9149 | AUPR: 0.5686 | Acc: 94.47%
best model(kernel=linear, C=0.1)
Valid:         AUC: 0.9151 | AUPR: 0.5683 | Acc: 94.52%
Internal test: AUC: 0.9159 | AUPR: 0.6239 | Acc: 94.57%
External test: AUC: 0.9158 | AUPR: 0.5604 | Acc: 94.25%


In [8]:
from sklearn.ensemble import GradientBoostingClassifier

model_fn = functools.partial(
    GradientBoostingClassifier, random_state=SEED,
    learning_rate=0.1, max_depth=5, loss='log_loss', 
    n_estimators=100)
param_grid_list = [{
        'criterion': ['friedman_mse', 'squared_error'],
    }
]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

GradientBoostingClassifier
model(criterion=friedman_mse)
Valid:         AUC: 0.9181 | AUPR: 0.6164 | Acc: 95.04%
model(criterion=squared_error)
Valid:         AUC: 0.9180 | AUPR: 0.6160 | Acc: 95.05%
best model(criterion=friedman_mse)
Valid:         AUC: 0.9181 | AUPR: 0.6164 | Acc: 95.04%
Internal test: AUC: 0.9197 | AUPR: 0.6646 | Acc: 95.06%
External test: AUC: 0.9180 | AUPR: 0.6337 | Acc: 94.98%


In [9]:
from sklearn.ensemble import AdaBoostClassifier
model_fn = functools.partial(AdaBoostClassifier, random_state=SEED, algorithm="SAMME")
param_grid_list = [{
    'n_estimators': [100, 200, 500],
    'learning_rate': [1.0],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

AdaBoostClassifier
model(n_estimators=100, learning_rate=1.0)
Valid:         AUC: 0.9153 | AUPR: 0.5960 | Acc: 94.85%
model(n_estimators=200, learning_rate=1.0)
Valid:         AUC: 0.9155 | AUPR: 0.5987 | Acc: 94.93%
model(n_estimators=500, learning_rate=1.0)
Valid:         AUC: 0.9148 | AUPR: 0.5961 | Acc: 94.96%
best model(n_estimators=200, learning_rate=1.0)
Valid:         AUC: 0.9155 | AUPR: 0.5987 | Acc: 94.93%
Internal test: AUC: 0.9124 | AUPR: 0.6432 | Acc: 94.84%
External test: AUC: 0.9130 | AUPR: 0.5798 | Acc: 94.58%


In [10]:
from sklearn.neighbors import KNeighborsClassifier

model_fn = functools.partial(
    KNeighborsClassifier, n_jobs=-1, algorithm='auto', p=2)
param_grid_list = [{
    'n_neighbors': [3, 5, 7],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

KNeighborsClassifier
model(n_neighbors=3)
Valid:         AUC: 0.7660 | AUPR: 0.3610 | Acc: 93.67%
model(n_neighbors=5)
Valid:         AUC: 0.7959 | AUPR: 0.4153 | Acc: 94.03%
model(n_neighbors=7)
Valid:         AUC: 0.8153 | AUPR: 0.4446 | Acc: 94.37%
best model(n_neighbors=7)
Valid:         AUC: 0.8153 | AUPR: 0.4446 | Acc: 94.37%
Internal test: AUC: 0.8335 | AUPR: 0.4980 | Acc: 94.36%
External test: AUC: 0.8241 | AUPR: 0.4707 | Acc: 94.30%


In [11]:
from sklearn.tree import DecisionTreeClassifier

model_fn = functools.partial(
    DecisionTreeClassifier, random_state=SEED,
    ccp_alpha=0.0, class_weight='balanced', max_depth=5)
param_grid_list = [{
        'criterion': ['gini', 'log_loss', 'entropy'],
    }
]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

DecisionTreeClassifier
model(criterion=gini)
Valid:         AUC: 0.8842 | AUPR: 0.5361 | Acc: 80.01%
model(criterion=log_loss)
Valid:         AUC: 0.8776 | AUPR: 0.5380 | Acc: 81.26%
model(criterion=entropy)
Valid:         AUC: 0.8776 | AUPR: 0.5380 | Acc: 81.26%
best model(criterion=gini)
Valid:         AUC: 0.8842 | AUPR: 0.5361 | Acc: 80.01%
Internal test: AUC: 0.8562 | AUPR: 0.5167 | Acc: 81.22%
External test: AUC: 0.8609 | AUPR: 0.4857 | Acc: 81.21%


In [12]:
from sklearn.naive_bayes import GaussianNB

model_fn = GaussianNB
param_grid_list = [{}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

GaussianNB
model()
Valid:         AUC: 0.8581 | AUPR: 0.3325 | Acc: 82.25%
best model()
Valid:         AUC: 0.8581 | AUPR: 0.3325 | Acc: 82.25%
Internal test: AUC: 0.8674 | AUPR: 0.3781 | Acc: 82.00%
External test: AUC: 0.8533 | AUPR: 0.3395 | Acc: 80.27%


In [13]:
dataframe = pd.DataFrame(results_dict).transpose().reset_index()
dataframe.columns = ['model', 'val_auc', 'val_aupr', 'val_acc', 'intest_auc', 'intest_aupr', 'intest_acc', 'extest_auc', 'extest_aupr', 'extest_acc']
# dataframe.to_csv()
filename = os.path.join(RESULTS_DIR, encode_method, impute_method, f'{fs_method}{int(fs_ratio * 142)}.csv')
dirname = os.path.dirname(filename)
if not os.path.exists(dirname):
    os.makedirs(dirname)
dataframe.to_csv(filename, index=False)